# CitiBike TSP
**Algorithms Group 50**\
Donal Lowsley-Williams \
Joan La Rosa \
Isaac Lichter \
Weidong Gao 

We will be conducting the traveling salesman problem on New York City citibike data. We will be using each CitiBike station as the nodes in our graph, and represent the edge costs by the average time it takes to bike from one station to another. Edges will only include those rides in the CitiBike data that have actually occurred - in other words, if no ride in the data started at station A and ended at station B, we will not include that edge. This adds to the realism of the problem as it would be unlikely to take that edge anyways. Moreover, we will only have edges from one station to another - not one station to itself, as there are occurences of rides like this in the CitiBike data.

### Imports
Will we be implementing these algorithms by hand, so we just need libraries for data loading and export, matrix manipulation functions, and some data exploration. 

In [1]:
import csv
import json
import numpy as np
import pandas as pd
import time

# The Data
We will get the data from CitiBike's openly available [System Data](https://s3.amazonaws.com/tripdata/index.html). The data we are using was the most recent as of this notebook's creation date, and was published by CitiBike on [**Oct 4th 2021, 01:26:29 pm**](https://s3.amazonaws.com/tripdata/202109-citibike-tripdata.csv.zip). This is the data of all rides taken on the CitiBike system during the month of September 2021. However, the preprocessing should work for any selection of the data from the system data, since they all should follow the same structure. 

Please store the data in a folder named **"Data"** with the name **"citibike-data.csv"**, like so: **"Data/citibike-data.csv**

In [4]:
df = pd.read_csv('Data/citibike-data.csv',
                 parse_dates=['started_at','ended_at'])

/Users/wdgao/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Preprocessing
In order to preprocess the data, 

### Data Cleaning

First lets take a look at the length of our data.

In [5]:
len(df)

3280221

Now, lets see what the column was that caused data type issues. We can see from the error above that it was column 7, which corresponds to the `end_station_id`. 

In [6]:
df.columns[7]

'end_station_id'

In the data, we can see by looking at the csv file that these values are typically float values, so we can cast them to a numeric value. We set any values that can't be cast to a numeric value to `NaN` by setting `errors='coerce'`. 

In [7]:
df['end_station_id'] = df['end_station_id'].apply(pd.to_numeric, errors='coerce')

We drop all null values. (those with `NaN`) This includes not just those with a faulty `end_station_id` value but those with errors elsewhere. We want to use only entries with full and complete data.

In [8]:
df = df.dropna()

We can see that we dropped approx. 15K entries that had data issues or invalid end stations. This is negligible in the grand scheme of the project, given that we have about 3.2M entries. 

In [9]:
len(df)

3264732

Let's take a peek at our data and its types. We can see that many use the dtype object, which pandas uses to refer to strings with varying length. We can leave this be. We have a couple date items, which we set when reading the csv file. The remaining data is all float data. 

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3264732 entries, 0 to 3280220
Data columns (total 13 columns):
ride_id               object
rideable_type         object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name    object
start_station_id      float64
end_station_name      object
end_station_id        float64
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual         object
dtypes: datetime64[ns](2), float64(6), object(5)
memory usage: 348.7+ MB


Let's conduct a quick sanity check to ensure the data is properly cleaned:

In [11]:
len(df['start_station_id'].unique()) == len(df['start_station_name'].unique())

False

Uh-oh. Lets dig into this to see what the problem is. Lets look at these values. 

In [12]:
print('start station id uniques: ',len(df['start_station_id'].unique()))
print('start station name uniques: ',len(df['start_station_name'].unique()))
print(len(df['start_station_id'].unique()) == len(df['start_station_name'].unique()))

start station id uniques:  1488
start station name uniques:  1493
False


So we see that there are more names than there are id's. Lets make a copy of the columns in the data causing an issue. 

In [13]:
sample = df[['start_station_id', 'start_station_name']].copy().drop_duplicates()

Next, we are going to pull each unique row by dropping the duplicates from this sample. This will let us examing all the unique start station id / name pairs. Next, we can count the frequency of each id occuring - since we know there are less id's than there are names. We can then sort this and view the id's that have more than one name.

In [14]:
sample.groupby('start_station_id').count().sort_values('start_station_name').tail(10)

,start_station_name
start_station_id,
5303.08,1
5303.06,1
5300.06,1
5300.05,1
8841.03,1
5382.07,2
4488.09,2
5190.09,2
5422.04,2


Now we see the problematic values are the last five entries in the above dataframe. We can iterate over these and observe what might be the problem by converting them to raw NumPy arrays. 

In [15]:
faulty_ids = np.array(
    sample.groupby('start_station_id').count().sort_values('start_station_name').tail(5).T.columns
)

# iterate over the faulty station ids
for idx, station_id in enumerate(faulty_ids):
    faulty_entries = sample.loc[sample['start_station_id'] == station_id]
    print('Entry',idx,np.array(faulty_entries['start_station_name']))
del sample, faulty_entries, faulty_ids, idx, station_id

Entry 0 ['Forsyth St\\t& Grand St' 'Forsyth St\t& Grand St']
Entry 1 ['Boerum Pl\\t& Pacific St' 'Boerum Pl\t& Pacific St']
Entry 2 ['Clinton St\\t& Cherry St' 'Clinton St\t& Cherry St']
Entry 3 ['Howard St & Lafayette St' 'Howard St & Centre St']
Entry 4 ['Nassau St\\t& Duffield St' 'Nassau St\t& Duffield St']


The problem for most of these appear to be some sort of string parsing issue. This is good, because it means we can ignore it and **simply focus on the station_id's**. However, we see that entry 3 has actual different names. Let us examing this in [Google Maps](https://www.google.com/maps/place/Citi+Bike:+Howard+St+%26+Lafayette+St/@40.7191002,-73.9996465,20z/data=!4m13!1m7!3m6!1s0x89c2598992b42c11:0xa58f248cc38ef667!2sLafayette+St+%26+Howard+St,+New+York,+NY+10013!3b1!8m2!3d40.7192055!4d-73.9998472!3m4!1s0x89c25989ed219c2b:0x5828092994c0b427!8m2!3d40.7191053!4d-73.9997333). We can see that this station is nestled right between those two streets, so it refers to the same station. Again, we can ignore this and **simply focus on the station_id's**.

![title](./lafayettecentre.png)

Let us perform one last sanity check before we start feature engineering, and ensure that there are the same number of start stations as there are end stations.

In [16]:
len(df['start_station_id'].unique()) == len(df['end_station_id'].unique())

True

We can ensure that all the entries in start_station_id are also present in end_station_id. This way, we know by iterating only over one of these lists ensures we are iterating across all potential station_ids. 

In [17]:
is_complete = True
for val in df['start_station_id'].unique():
    if val not in df['end_station_id'].unique():
        is_complete = False
print('data matches') if is_complete else print('Error; data mistmatch')
del val, is_complete

data matches


Looks like we are good to go. Let's pull the necesary data for our algorithms into another dataframe to keep things clean.

### Edge Costs
Now we need to compute edge costs between each station, and store this in a cost matrix. We can do this by first simply subtracting the start time from the end time. Since these are date objects, we can convert them to raw integers representing seconds by dividing by a 1 second numpy date time value to get total seconds and then casting to int32.

In [18]:
df['trip_cost'] = ((df['ended_at'] - df['started_at']) / np.timedelta64(1, 's')).astype(np.int32)

Next up is creating our actual cost data. What we want here is a DataFrame that contains average trip cost from each start station to each end station (when they are not the same), across all rides between those two stations. We can do this by pulling out only our relevant values - `start_station_id`, `end_station_id`, and `trip_cost`, and then dropping all the entries where the start station is equivalent to the end station. Then we group them by their `start_station_id` / `end_station_id` and take an average of the remaining columns - which is just `trip_cost`.

In [19]:
new_df = df[['start_station_id', 'end_station_id', 'trip_cost']]
dropped = new_df.drop(new_df[new_df['start_station_id'] == new_df['end_station_id']].index)
edge_costs = dropped.groupby(by=['start_station_id','end_station_id']).mean().reset_index()
del dropped, new_df;

Lets take a look at our edge cost DataFrame.

In [20]:
edge_costs

,start_station_id,end_station_id,trip_cost
0,2733.03,2782.02,674.857143
1,2733.03,2832.03,1108.142857
2,2733.03,2872.02,651.750000
3,2733.03,2883.03,466.666667
4,2733.03,2912.08,692.333333
...,...,...,...
406129,8841.03,8782.01,1353.250000
406130,8841.03,8795.01,1514.631579
406131,8841.03,8795.03,843.333333
406132,8841.03,8799.01,1213.818182


As we can see, we have each `start_station_id`, each `end_station_id`, and the average `trip_cost` between those stations. We are now prepared to run our algorithms. 

# The Algorithms
Now that we have pulled and preprocessed our data, we can begin to test various algorithms against it. We will test the following algorithms:

**Greedy**
- Nearest Neighbor
- Farthest First Traversal

**?????**
- Christofides Algorithm 

**K-opt Heuristic Algorithms**
- 2-OPT
- 3-OPT
- Lin-Kernighan Heuristic

### Helper Function
Before we do anything, let's write a little function to time our algorithms and make for easy comparison. 

In [ ]:
def check_performance(fn, station_ids, distances):
    start = time.time()
    _, _ = fn(station_ids, distances)
    end = time.time()
    return end - start

def compare_algorithms(fn_list, station_ids, distances, num_iters=5):
    results = {}
    for algo in fn_list:
        t = 0
        for _ in range(num_iters):
            t += check_performance(algo, station_ids, distances)
        results[algo.__name__] = t / num_iters
    return results

## Greedy Algorithms
First, we will examine various greedy algorithms and their effectiveness on the data. 

### Nearest Neighbor

The nearest neighbor algorithm is very simple and simply starts at a random node (or station in our example) and continues to visit unvisited nodes that are as close to the last visited node as possible .

**Psuedocode:** \
These are the steps of the algorithm:

    Initialize all vertices as unvisited.
    Select an arbitrary vertex, set it as the current vertex u. Mark u as visited.
    Find out the shortest edge connecting the current vertex u and an unvisited vertex v.
    Set v as the current vertex u. Mark v as visited.
    If all the vertices in the domain are visited, then terminate. Else, go to step 3.

The sequence of the visited vertices is the output of the algorithm. 

**Time complexity:** Worst Case O(N^2)

**Space complexity:** Worst Case O(N)

[Source (wikipedia)](https://en.wikipedia.org/wiki/Nearest_neighbour_algorithm)

In [ ]:
# Nearest Neighbor
def nearest_neighbor(edge_costs : pd.DataFrame):
    cost = 0
    path = []
    
    return path, cost

### Farthest-First Traversal

**PseudoCode:**\
The farthest-first traversal of a finite point set may be computed by a greedy algorithm that maintains the distance of each point from the previously selected points, performing the following steps:[3]

    Initialize the sequence of selected points to the empty sequence, and the distances of each point to the selected points to infinity.
    While not all points have been selected, repeat the following steps:
        Scan the list of not-yet-selected points to find a point p that has maximum distance from the selected points.
        Remove p from the not-yet-selected points and add it to the end of the sequence of selected points.
        For each remaining not-yet-selected point q, replace the distance stored for q by the minimum of its old value and the distance from p to q.

**Time complexity:** Worst Case O(N^2)

**Space complexity:** Worst Case O(N)

[Source (wikipedia)](https://en.wikipedia.org/wiki/Farthest-first_traversal)

In [ ]:
# Farthest-First Traversal

### K-opt Agorithms

#### 2-opt Algorithm

The 2-opt algorithm works as follows: take 2 arcs from the route, reconnect these arcs with each other and calculate new travel distance. If this modification has led to a shorter total travel distance the current route is updated. The algorithm continues to build on the improved route and repeats the steps.

```
procedure 2optSwap(route, i, k) {
    1. take route[0] to route[i-1] and add them in order to new_route
    2. take route[i] to route[k] and add them in reverse order to new_route
    3. take route[k+1] to end and add them in order to new_route
    return new_route;
}
```

In [2]:
def two_opt(route):
    # 2-opt implementation here
    return

In [35]:
edge_costs

,start_station_id,end_station_id,trip_cost
0,2733.03,2782.02,674.857143
1,2733.03,2832.03,1108.142857
2,2733.03,2872.02,651.750000
3,2733.03,2883.03,466.666667
4,2733.03,2912.08,692.333333
...,...,...,...
406129,8841.03,8782.01,1353.250000
406130,8841.03,8795.01,1514.631579
406131,8841.03,8795.03,843.333333
406132,8841.03,8799.01,1213.818182


In [104]:
print(get_edge_cost(edge_costs, 2733.03, 2782.02))
print(get_edge_cost(edge_costs, 2782.02, 2733.03))

674.8571428571429
761.0666666666667


In [142]:
stations = df['start_station_id'].unique()

In [143]:
stations

array([7141.07, 5175.08, 3651.04, ..., 8290.01, 4203.04, 8285.11])

In [144]:
def get_edge_cost(edge_costs, start_station_id, end_station_id):
    cost = edge_costs[(edge_costs["start_station_id"] == start_station_id) \
                      & (edge_costs["end_station_id"] == end_station_id)]
    if cost.empty:
        return 10000
    else:
        return cost.get("trip_cost").iloc[0]


In [146]:
import matplotlib.pyplot as plt

MAXCOUNT = 20

def calPathDist(edge_costs, path):
    sum = 0.0
    for i in range(1, len(path)):
        sum += get_edge_cost(edge_costs, path[i], path[i - 1])
    return sum    

def pathCompare(path1, path2):
    if calPathDist(edge_costs, path1) <= calPathDist(edge_costs, path2):
        return True
    return False
    
def generateRandomPath(path):
    a = np.random.randint(len(path))
    while True:
        b = np.random.randint(len(path))
        if np.abs(a - b) > 1:
            break
    if a > b:
        return b, a, path[b:a+1]
    else:
        return a, b, path[a:b+1]
    
def reversePath(path):
    rePath = path.copy()
    rePath[1:-1] = rePath[-2:0:-1]
    return rePath
    
def updateBestPath(bestPath):
    count = 0
    while count < MAXCOUNT:
        print("updated path costs to", calPathDist(edge_costs, bestPath))
#         print(bestPath.tolist())
        start, end, path = generateRandomPath(bestPath)
        rePath = reversePath(path)
        if pathCompare(path, rePath):
            count += 1
            continue
        else:
            count = 0
            bestPath[start:end+1] = rePath
    return bestPath
    
def two_opt(stations):
    bestPath = df['start_station_id'].unique()
    bestPath = np.append(bestPath, stations[0])
    bestPath = updateBestPath(bestPath)
    
two_opt(stations)


#### 3-opt Algorithm
3-opt analysis involves deleting 3 connections (or edges) in a network (or tour), to create 3 sub-tours. Then the 7 different ways of reconnecting the network are analysed to find the optimum one. This process is then repeated for a different set of 3 connections, until all possible combinations have been tried in a network. A single execution of 3-opt has a time complexity of $O(n^{3})$.


In [ ]:
def three_opt(route):
    # 3-opt implementation here
    return

In [164]:
def exchange(path, mode, a, c, e):
    """
    path: sequence of station_id with i th being starting and i + 1 th being ending staions
    mode: mode of exchanging
    a, c, e: index of stations
    """
    b, d, f = a + 1, c + 1, e + 1
    
    
    
    if mode == 0:
        # remain unchanged
        return path, 0
    elif mode == 1:
        # 2-opt (a, e)[d, c](b, f)
        sol = np.concatenate((path[: a + 1], path[e: d - 1: -1], path[c: b - 1: -1], path[f:]))
    elif mode == 2:
        # 2-opt [a, c](b, d)[e, f]
        sol = np.concatenate((path[: a + 1], path[b: c + 1], path[e:d - 1: -1], path[f:]))
    elif mode == 3:
        # 2-opt (a, c)(b, d)[e, f]
        sol = np.concatenate((path[: a + 1], path[c: b - 1: -1], path[d: e + 1], path[f:]))
    elif mode == 4:
        # 3-opt (a, c)(b, e)(d, f)
        sol = np.concatenate((path[: a + 1], path[c: b - 1: -1], path[e: d - 1: -1], path[f:]))
    elif mode == 5:
        # 3-opt (a, e)(d, b)(c, f)
        sol = np.concatenate((path[: a + 1], path[e: d - 1: -1], path[b: c + 1], path[f:]))
    elif mode == 6:
        # 3-opt (a, d)(e, c)(b, f)
        sol = np.concatenate((path[: a + 1], path[d: e + 1], path[c: b - 1: -1], path[f:]))
    elif mode == 7:
        # 3-opt (a, d)(e, b)(c, f)
        sol = np.concatenate((path[: a + 1], path[d: e + 1], path[b: c + 1], path[f:]))
    
    base = calPathDist(edge_costs, path)
    newCost = calPathDist(edge_costs, sol)
    return sol, base - newCost

def three_opt_search(bestPath):
    size = len(bestPath)
    bestCost = calPathDist(edge_costs, bestPath)
    bestChange = 1
    
    while bestChange > 0:
        saved, bestChange = three_opt_update(bestPath, size)
        print("current route cost is", bestCost)
        if bestChange > 0:
            a, c, e, mode = saved
            bestPath, change = exchange(bestPath, mode, a, c, e)
            bestCost -= change

def three_opt_update(bestPath, size):
    bestChange = 0
    saved = None
    for a in range(size - 5):
        for c in range(a + 2, size - 3):
            for e in range(c + 2, size - 1):
                change = 0
                print(a, c, e, change, bestChange)
                for i in range(8):
                    path, change = exchange(bestPath, i, a, c, e)
                    
                    if change > bestChange:
                        saved = a, c, e, i
                        bestChange = change
    return saved, bestChange
        
        

In [165]:
import random
import numpy as np
import matplotlib.pyplot as plt
# import TWOPT
from itertools import combinations, permutations

def three_opt(stations):
    bestPath = stations
    bestPath = np.append(bestPath, stations[0])
    bestPath = three_opt_search(bestPath)

In [ ]:
three_opt(stations[:100])

0 2 4 0 0
0 2 5 0 9755.147619047668
0 2 6 0 9755.147619047668
0 2 7 0 10131.066666666651
0 2 8 0 10131.066666666651
0 2 9 0 10131.066666666651
0 2 10 0 10131.066666666651
0 2 11 0 10131.066666666651
0 2 12 0 10131.066666666651
0 2 13 0 10131.066666666651
0 2 14 0 10131.066666666651
0 2 15 0 10131.066666666651
0 2 16 0 10131.066666666651
0 2 17 0 11993.064358974341
0 2 18 0 12573.231025641086
0 2 19 0 28931.421501831384
0 2 20 0 28931.421501831384
0 2 21 0 28931.421501831384
0 2 22 0 28931.421501831384
0 2 23 0 28931.421501831384
0 2 24 0 28931.421501831384
0 2 25 0 28931.421501831384
0 2 26 0 28931.421501831384
0 2 27 0 28931.421501831384
0 2 28 0 28931.421501831384
0 2 29 0 28931.421501831384
0 2 30 0 28931.421501831384
0 2 31 0 28931.421501831384
0 2 32 0 28931.421501831384
0 2 33 0 28931.421501831384
0 2 34 0 29947.06435897434
0 2 35 0 29947.06435897434
0 2 36 0 29947.06435897434
0 2 37 0 29947.06435897434
0 2 38 0 29947.06435897434
0 2 39 0 29947.06435897434
0 2 40 0 29947.06435897

0 5 26 0 37588.87864468852
0 5 27 0 37588.87864468852
0 5 28 0 37588.87864468852
0 5 29 0 37588.87864468852
0 5 30 0 37588.87864468852
0 5 31 0 37588.87864468852
0 5 32 0 37588.87864468852
0 5 33 0 37588.87864468852
0 5 34 0 37588.87864468852
0 5 35 0 37588.87864468852
0 5 36 0 37588.87864468852
0 5 37 0 37588.87864468852
0 5 38 0 37588.87864468852
0 5 39 0 37588.87864468852
0 5 40 0 37588.87864468852
0 5 41 0 37588.87864468852
0 5 42 0 37588.87864468852
0 5 43 0 37588.87864468852
0 5 44 0 37588.87864468852
0 5 45 0 37588.87864468852
0 5 46 0 37588.87864468852
0 5 47 0 37588.87864468852
0 5 48 0 37588.87864468852
0 5 49 0 37588.87864468852
0 5 50 0 37588.87864468852
0 5 51 0 37588.87864468852
0 5 52 0 37588.87864468852
0 5 53 0 37588.87864468852
0 5 54 0 37588.87864468852
0 5 55 0 37588.87864468852
0 5 56 0 37588.87864468852
0 5 57 0 37588.87864468852
0 5 58 0 37588.87864468852
0 5 59 0 37588.87864468852
0 5 60 0 37588.87864468852
0 5 61 0 37588.87864468852
0 5 62 0 37588.87864468852
0

0 8 57 0 37588.87864468852
0 8 58 0 37588.87864468852
0 8 59 0 37588.87864468852
0 8 60 0 37588.87864468852
0 8 61 0 37588.87864468852
0 8 62 0 37588.87864468852
0 8 63 0 37588.87864468852
0 8 64 0 37588.87864468852
0 8 65 0 37588.87864468852
0 8 66 0 37588.87864468852
0 8 67 0 37588.87864468852
0 8 68 0 37588.87864468852
0 8 69 0 37588.87864468852
0 8 70 0 37588.87864468852
0 8 71 0 37588.87864468852
0 8 72 0 37588.87864468852
0 8 73 0 37588.87864468852
0 8 74 0 37588.87864468852
0 8 75 0 37588.87864468852
0 8 76 0 37588.87864468852
0 8 77 0 37588.87864468852
0 8 78 0 37588.87864468852
0 8 79 0 37588.87864468852
0 8 80 0 37588.87864468852
0 8 81 0 37588.87864468852
0 8 82 0 37588.87864468852
0 8 83 0 37588.87864468852
0 8 84 0 37588.87864468852
0 8 85 0 37588.87864468852
0 8 86 0 37588.87864468852
0 8 87 0 37588.87864468852
0 8 88 0 37588.87864468852
0 8 89 0 37588.87864468852
0 8 90 0 37588.87864468852
0 8 91 0 37588.87864468852
0 8 92 0 37588.87864468852
0 8 93 0 37588.87864468852
0

0 11 91 0 37588.87864468852
0 11 92 0 37588.87864468852
0 11 93 0 37588.87864468852
0 11 94 0 37588.87864468852
0 11 95 0 37588.87864468852
0 11 96 0 37588.87864468852
0 11 97 0 37588.87864468852
0 11 98 0 37588.87864468852
0 11 99 0 37588.87864468852
0 12 14 0 37588.87864468852
0 12 15 0 37588.87864468852
0 12 16 0 37588.87864468852
0 12 17 0 37588.87864468852
0 12 18 0 37588.87864468852
0 12 19 0 37588.87864468852
0 12 20 0 37588.87864468852
0 12 21 0 37588.87864468852
0 12 22 0 37588.87864468852
0 12 23 0 37588.87864468852
0 12 24 0 37588.87864468852
0 12 25 0 37588.87864468852
0 12 26 0 37588.87864468852
0 12 27 0 37588.87864468852
0 12 28 0 37588.87864468852
0 12 29 0 37588.87864468852
0 12 30 0 37588.87864468852
0 12 31 0 37588.87864468852
0 12 32 0 37588.87864468852
0 12 33 0 37588.87864468852
0 12 34 0 37588.87864468852
0 12 35 0 37588.87864468852
0 12 36 0 37588.87864468852
0 12 37 0 37588.87864468852
0 12 38 0 37588.87864468852
0 12 39 0 37588.87864468852
0 12 40 0 37588.8786

0 15 46 0 37588.87864468852
0 15 47 0 37588.87864468852
0 15 48 0 37588.87864468852
0 15 49 0 37588.87864468852
0 15 50 0 37588.87864468852
0 15 51 0 37588.87864468852
0 15 52 0 37588.87864468852
0 15 53 0 37588.87864468852
0 15 54 0 37588.87864468852
0 15 55 0 37588.87864468852
0 15 56 0 37588.87864468852
0 15 57 0 37588.87864468852
0 15 58 0 37588.87864468852
0 15 59 0 37588.87864468852
0 15 60 0 37588.87864468852
0 15 61 0 37588.87864468852
0 15 62 0 37588.87864468852
0 15 63 0 37588.87864468852
0 15 64 0 37588.87864468852
0 15 65 0 37588.87864468852
0 15 66 0 37588.87864468852
0 15 67 0 37588.87864468852
0 15 68 0 37588.87864468852
0 15 69 0 37588.87864468852
0 15 70 0 37588.87864468852
0 15 71 0 37588.87864468852
0 15 72 0 37588.87864468852
0 15 73 0 37588.87864468852
0 15 74 0 37588.87864468852
0 15 75 0 37588.87864468852
0 15 76 0 37588.87864468852
0 15 77 0 37588.87864468852
0 15 78 0 37588.87864468852
0 15 79 0 37588.87864468852
0 15 80 0 37588.87864468852
0 15 81 0 37588.8786

...

6 70 82 0 46625.56435897434
6 70 83 0 46625.56435897434
6 70 84 0 46625.56435897434
6 70 85 0 46625.56435897434
6 70 86 0 46625.56435897434
6 70 87 0 46625.56435897434
6 70 88 0 46625.56435897434
6 70 89 0 46625.56435897434
6 70 90 0 46625.56435897434
6 70 91 0 46625.56435897434
6 70 92 0 46625.56435897434
6 70 93 0 46625.56435897434
6 70 94 0 46625.56435897434
6 70 95 0 46625.56435897434
6 70 96 0 46625.56435897434
6 70 97 0 46625.56435897434
6 70 98 0 46625.56435897434
6 70 99 0 46625.56435897434
6 71 73 0 46625.56435897434
6 71 74 0 46625.56435897434
6 71 75 0 46625.56435897434
6 71 76 0 46625.56435897434
6 71 77 0 46625.56435897434
6 71 78 0 46625.56435897434
6 71 79 0 46625.56435897434
6 71 80 0 46625.56435897434
6 71 81 0 46625.56435897434
6 71 82 0 46625.56435897434
6 71 83 0 46625.56435897434
6 71 84 0 46625.56435897434
6 71 85 0 46625.56435897434
6 71 86 0 46625.56435897434
6 71 87 0 46625.56435897434
6 71 88 0 46625.56435897434
6 71 89 0 46625.56435897434
6 71 90 0 46625.5643

6 84 88 0 46625.56435897434
6 84 89 0 46625.56435897434
6 84 90 0 46625.56435897434
6 84 91 0 46625.56435897434
6 84 92 0 46625.56435897434
6 84 93 0 46625.56435897434
6 84 94 0 46625.56435897434
6 84 95 0 46625.56435897434
6 84 96 0 46625.56435897434
6 84 97 0 46625.56435897434
6 84 98 0 46625.56435897434
6 84 99 0 46625.56435897434
6 85 87 0 46625.56435897434
6 85 88 0 46625.56435897434
6 85 89 0 46625.56435897434
6 85 90 0 46625.56435897434
6 85 91 0 46625.56435897434
6 85 92 0 46625.56435897434
6 85 93 0 46625.56435897434
6 85 94 0 46625.56435897434
6 85 95 0 46625.56435897434
6 85 96 0 46625.56435897434
6 85 97 0 46625.56435897434
6 85 98 0 46625.56435897434
6 85 99 0 46625.56435897434
6 86 88 0 46625.56435897434
6 86 89 0 46625.56435897434
6 86 90 0 46625.56435897434
6 86 91 0 46625.56435897434
6 86 92 0 46625.56435897434
6 86 93 0 46625.56435897434
6 86 94 0 46625.56435897434
6 86 95 0 46625.56435897434
6 86 96 0 46625.56435897434
6 86 97 0 46625.56435897434
6 86 98 0 46625.5643

7 11 29 0 46625.56435897434
7 11 30 0 46625.56435897434
7 11 31 0 46625.56435897434
7 11 32 0 46625.56435897434
7 11 33 0 46625.56435897434
7 11 34 0 46625.56435897434
7 11 35 0 46625.56435897434
7 11 36 0 46625.56435897434
7 11 37 0 46625.56435897434
7 11 38 0 46625.56435897434
7 11 39 0 46625.56435897434
7 11 40 0 46625.56435897434
7 11 41 0 46625.56435897434
7 11 42 0 46625.56435897434
7 11 43 0 46625.56435897434
7 11 44 0 46625.56435897434
7 11 45 0 46625.56435897434
7 11 46 0 46625.56435897434
7 11 47 0 46625.56435897434
7 11 48 0 46625.56435897434
7 11 49 0 46625.56435897434
7 11 50 0 46625.56435897434
7 11 51 0 46625.56435897434
7 11 52 0 46625.56435897434
7 11 53 0 46625.56435897434
7 11 54 0 46625.56435897434
7 11 55 0 46625.56435897434
7 11 56 0 46625.56435897434
7 11 57 0 46625.56435897434
7 11 58 0 46625.56435897434
7 11 59 0 46625.56435897434
7 11 60 0 46625.56435897434
7 11 61 0 46625.56435897434
7 11 62 0 46625.56435897434
7 11 63 0 46625.56435897434
7 11 64 0 46625.5643

7 14 67 0 46625.56435897434
7 14 68 0 46625.56435897434
7 14 69 0 46625.56435897434
7 14 70 0 46625.56435897434
7 14 71 0 46625.56435897434
7 14 72 0 46625.56435897434
7 14 73 0 46625.56435897434
7 14 74 0 46625.56435897434
7 14 75 0 46625.56435897434
7 14 76 0 46625.56435897434
7 14 77 0 46625.56435897434
7 14 78 0 46625.56435897434
7 14 79 0 46625.56435897434
7 14 80 0 46625.56435897434
7 14 81 0 46625.56435897434
7 14 82 0 46625.56435897434
7 14 83 0 46625.56435897434
7 14 84 0 46625.56435897434
7 14 85 0 46625.56435897434
7 14 86 0 46625.56435897434
7 14 87 0 46625.56435897434
7 14 88 0 46625.56435897434
7 14 89 0 46625.56435897434
7 14 90 0 46625.56435897434
7 14 91 0 46625.56435897434
7 14 92 0 46625.56435897434
7 14 93 0 46625.56435897434
7 14 94 0 46625.56435897434
7 14 95 0 46625.56435897434
7 14 96 0 46625.56435897434
7 14 97 0 46625.56435897434
7 14 98 0 46625.56435897434
7 14 99 0 46625.56435897434
7 15 17 0 46625.56435897434
7 15 18 0 46625.56435897434
7 15 19 0 46625.5643

7 18 34 0 46625.56435897434
7 18 35 0 46625.56435897434
7 18 36 0 46625.56435897434
7 18 37 0 46625.56435897434
7 18 38 0 46625.56435897434
7 18 39 0 46625.56435897434
7 18 40 0 46625.56435897434
7 18 41 0 46625.56435897434
7 18 42 0 46625.56435897434
7 18 43 0 46625.56435897434
7 18 44 0 46625.56435897434
7 18 45 0 46625.56435897434
7 18 46 0 46625.56435897434
7 18 47 0 46625.56435897434
7 18 48 0 46625.56435897434
7 18 49 0 46625.56435897434
7 18 50 0 46625.56435897434
7 18 51 0 46625.56435897434
7 18 52 0 46625.56435897434
7 18 53 0 46625.56435897434
7 18 54 0 46625.56435897434
7 18 55 0 46625.56435897434
7 18 56 0 46625.56435897434
7 18 57 0 46625.56435897434
7 18 58 0 46625.56435897434
7 18 59 0 46625.56435897434
7 18 60 0 46625.56435897434
7 18 61 0 46625.56435897434
7 18 62 0 46625.56435897434
7 18 63 0 46625.56435897434
7 18 64 0 46625.56435897434
7 18 65 0 46625.56435897434
7 18 66 0 46625.56435897434
7 18 67 0 46625.56435897434
7 18 68 0 46625.56435897434
7 18 69 0 46625.5643

In [ ]:
def tsp(data):
    # build a graph
    G = build_graph(data)
    print("Graph: ", G)

    # build a minimum spanning tree
    MSTree = minimum_spanning_tree(G)
    print("MSTree: ", MSTree)

    # find odd vertexes
    odd_vertexes = find_odd_vertexes(MSTree)
    print("Odd vertexes in MSTree: ", odd_vertexes)

    # add minimum weight matching edges to MST
    minimum_weight_matching(MSTree, G, odd_vertexes)
    print("Minimum weight matching: ", MSTree)

    # find an eulerian tour
    eulerian_tour = find_eulerian_tour(MSTree, G)

    print("Eulerian tour: ", eulerian_tour)

    current = eulerian_tour[0]
    path = [current]
    visited = [False] * len(eulerian_tour)
    visited[0] = True

    length = 0

    for v in eulerian_tour[1:]:
        if not visited[v]:
            path.append(v)
            visited[v] = True

            length += G[current][v]
            current = v

    path.append(path[0])

    print("Result path: ", path)
    print("Result length of the path: ", length)

    return length, path


def get_length(x1, y1, x2, y2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** (1.0 / 2.0)


def build_graph(data):
    graph = {}
    for this in range(len(data)):
        for another_point in range(len(data)):
            if this != another_point:
                if this not in graph:
                    graph[this] = {}

                graph[this][another_point] = get_length(data[this][0], data[this][1], data[another_point][0],
                                                        data[another_point][1])

    return graph


class UnionFind:
    def __init__(self):
        self.weights = {}
        self.parents = {}

    def __getitem__(self, object):
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root

    def __iter__(self):
        return iter(self.parents)

    def union(self, *objects):
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r], r) for r in roots])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest


def minimum_spanning_tree(G):
    tree = []
    subtrees = UnionFind()
    for W, u, v in sorted((G[u][v], u, v) for u in G for v in G[u]):
        if subtrees[u] != subtrees[v]:
            tree.append((u, v, W))
            subtrees.union(u, v)

    return tree


def find_odd_vertexes(MST):
    tmp_g = {}
    vertexes = []
    for edge in MST:
        if edge[0] not in tmp_g:
            tmp_g[edge[0]] = 0

        if edge[1] not in tmp_g:
            tmp_g[edge[1]] = 0

        tmp_g[edge[0]] += 1
        tmp_g[edge[1]] += 1

    for vertex in tmp_g:
        if tmp_g[vertex] % 2 == 1:
            vertexes.append(vertex)

    return vertexes


def minimum_weight_matching(MST, G, odd_vert):
    import random
    random.shuffle(odd_vert)

    while odd_vert:
        v = odd_vert.pop()
        length = float("inf")
        u = 1
        closest = 0
        for u in odd_vert:
            if v != u and G[v][u] < length:
                length = G[v][u]
                closest = u

        MST.append((v, closest, length))
        odd_vert.remove(closest)


def find_eulerian_tour(MatchedMSTree, G):
    # find neigbours
    neighbours = {}
    for edge in MatchedMSTree:
        if edge[0] not in neighbours:
            neighbours[edge[0]] = []

        if edge[1] not in neighbours:
            neighbours[edge[1]] = []

        neighbours[edge[0]].append(edge[1])
        neighbours[edge[1]].append(edge[0])

    # print("Neighbours: ", neighbours)

    # finds the hamiltonian circuit
    start_vertex = MatchedMSTree[0][0]
    EP = [neighbours[start_vertex][0]]

    while len(MatchedMSTree) > 0:
        for i, v in enumerate(EP):
            if len(neighbours[v]) > 0:
                break

        while len(neighbours[v]) > 0:
            w = neighbours[v][0]

            remove_edge_from_matchedMST(MatchedMSTree, v, w)

            del neighbours[v][(neighbours[v].index(w))]
            del neighbours[w][(neighbours[w].index(v))]

            i += 1
            EP.insert(i, w)

            v = w

    return EP


def remove_edge_from_matchedMST(MatchedMST, v1, v2):

    for i, item in enumerate(MatchedMST):
        if (item[0] == v2 and item[1] == v1) or (item[0] == v1 and item[1] == v2):
            del MatchedMST[i]

    return MatchedMST